# INF0417 - Visão computacional
## Lab3 - Rastreamento visual

# 1. Introdução

Neste laboratório você implementará a versão original do famoso algoritmo para rastreamento de features de Lucas-Kanade, o **LK-tracker**. \
A menos que seja implementado com operações de baixo nível, o LK-tracker se torna computacionalmente restritivo se tentarmos rastrear cada píxel na imagem. \
Assim, nesta atividade vamos restringir a tarefa de rastreamento a somente uma única região por vez (centrada ao redor de um píxel). \
Isso significa que a escolha do píxel a ser rastreado deve ser feita *"sabiamente"*. \
Além disso, devido à sua estreita relação com o LK-tracker, também incorporaremos o **detector de Harris** (*aka.*, Harris-Stephens detector) para encontrar *"automaticamente"* features *"boas"* de serem rastreadas.

## 1.1 Preparação

Como preparação para este laboratório, você deverá ler este guia e completar as atividades marcadas com **estrela (*)**. \
Parte da preparação consiste em ler e entender o artigo simplificado do rastreador LK-tracker ["Derivation of the Lucas-Kanade Tracker"](./references/2007%20-%20LK-derivation.pdf) \[1\]. \
Vamos a utilizar a mesma notação do artigo ao fazer perguntas sobre uma expressão em específico ou uma variável, *e.g.*, $T$. \
Para completar os exercícios a tempo, é importante estar familiarizado com os diferentes passos envolvidos no algoritmo de rastreamento LK-tracker.

O laboratório será implementado em Python Notebook (não use Google colab!). \
Na sala de laboratório do INF, você pode executar o Python desde o prompt no shell do `Ubuntu App`. \
Use `python3 <meu_script>.py` para executar um script Python ou, preferencialmente, trabalhe com uma IDE como `vscode` ou `pycharm`. \
Se você utiliza múltiplos scripts (recomendado) poderá salvar e carregar dados utilizando o pacote `pickle` ou `np.save/load`. \
Para um trabalho mais interativo, você também pode usar `ipython3`, que permite auto-completar comandos com tabs e também conta com  destaque de sintaxe.

Adicionando `from INF0417_labs import lab3` nos seus scripts, você poderá usar o código de suporte para este laboratório. \
Este código inclui funções auxiliares, tais como `lab3.load_lab_image` e `lab3.get_cameraman`. \
Use `help(lab3)` para obter uma descrição deste pacote.

**Dica**: Bugs, em códigos científicos, são fáceis de introduzir, mas difíceis de achar. \
São práticas recomendadas:
- Ler a documentação das funções que você usa e;
- Escrever códigos de teste que verifique que elas se comportem conforme o esperado.

**Preste atenção** para não confundir as indexações $(x,y)$ e $(\text{linha}, \text{coluna})$, pois você provavelmente usará ambas neste laboratório. \
Em laboratórios futuros, usaremos a $nD$-indexação, com exponencialmente mais maneiras de cometer tais erros.

In [ ]:
from INF0417_labs import lab3
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Mapa de cores verde-preto-vermelho para os gradientes
gkr_col = np.zeros((255, 4))
gkr_col[:, 3] = 1
gkr_col[:127, 1] = np.linspace(1.0, 0.0, 127, False)
gkr_col[127:, 0] = np.linspace(0.0, 1.0, 128, True)
gkr_col = ListedColormap(gkr_col)

# cameraman
I, J, dTrue = lab3.get_cameraman()
plt.figure()
plt.imshow(I, cmap='gray')
plt.show()

# cornertest
img = lab3.load_lab_image('cornertest.png')
plt.figure()
plt.imshow(img, cmap='gray')
plt.show()

# 2. Esboço do algoritmo

Como preparativos, começaremos este laboratório tendo uma estratégia clara sobre como abordar o problema de rastreamento.

**\* Pergunta de preparação**: Desenhe um diagrama de blocos que sintetize os principais passos do algoritmo LK-tracker.

In [ ]:
# Escreva aqui sua resposta

**\* Pergunta de preparação**: Escreva a equação final requerida para atualização do deslocamento.

In [ ]:
# Escreva aqui sua resposta

**\* Pergunta de preparação**: Usando a notação em \[1\], explique o que é $T$?

In [ ]:
# Escreva aqui sua resposta

**\* Pergunta de preparação**: O que é requerido de $T$, para que a equação final seja resolvida?

In [ ]:
# Escreva aqui sua resposta

**\* Pergunta de preparação**: Dado o seu entendimento sobre o [problema da apertura](https://youtu.be/vVGorOxMh8w) ([vídeo offline](./videos/aperture-problem.mp4)) e a resposta à pergunta anterior, o que deveria caracterizar um píxel (ou região) escolhido para o rastreamento?

In [ ]:
# Escreva aqui sua resposta

# 3. O rastreador de Lucas-Kanade

Nesta seção você implementará os blocos necessários para crear seu LK-tracker.

## 3.1 Gradiente regularizado

Começamos estimando todas as incógnitas na equação do LK-tracker. \
É útil ter uma função que estime todas elas para garantir que todas tenham a mesma regularização. \
Implemente uma função que tenha como entradas o tamanho do filtro e um desvio padrão e, retorne o *gradiente regularizado* e imagens compatíveis. \
Uma possível função protótipo é:

`gradiente_regularizado(I, J, kernel_size, sigma) -> Ig, Jg, Jgdx, Jgdy`

Entradas:
- `I`          : Imagem $I$
- `J`          : Imagem $J$
- `kernel_size`: Tamanho do filtro $g(\sigma)$ de ordem $N=$ `kernel_size`
- `sigma`      : Desvio padrão de $g(\sigma)$

Saída:
- `Ig`  : $I * g(\sigma)$
- `Jg`  : $J * g(\sigma)$
- `Jgdx`: $\frac{\partial }{\partial x}(J * g(\sigma))$
- `Jgdy`: $\frac{\partial }{\partial y}(J * g(\sigma))$

**Dica**: Isso foi discutido no laboratório [`INF0417_python-tutorial.ipynb`](https://github.com/aldodiaz-UFG/INF0417/blob/main/lab01_tutorial-python/lab01_python-tutorial.ipynb) e em \[1\].

In [ ]:
# Escreva aqui sua resposta
from scipy.signal import convolve2d as conv2

def gradiente_regularizado(I, J, kernel_size, sigma):
    ...
    
    Ig   = conv2(conv2(I, g, mode='same'), g.T, mode='same') # Imagem filtrada I * g
    Jg   = conv2(conv2(J, g, mode='same'), g.T, mode='same') # Imagem filtrada J * g
    Jgdx = conv2(conv2(J, df, mode='same'), g.T, mode='same') # Gradiente regularizado de J na horizontal
    Jgdy = conv2(conv2(J, df.T, mode='same'), g, mode='same') # Gradiente regularizado de J na vertical
    
    return Ig, Jg, Jgdx, Jgdy

In [ ]:
kernel_size = 15
sigma = 0.5
Ig, Jg, Jgdx, Jgdy = gradiente_regularizado(I, J, kernel_size, sigma)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 4, 1)
plt.imshow(Ig, cmap='gray')
plt.subplot(1, 4, 2)
plt.imshow(Jg, cmap='gray')
plt.subplot(1, 4, 3)
plt.imshow(Jgdx, vmin=-100, vmax=100, cmap=gkr_col)
plt.subplot(1, 4, 4)
plt.imshow(Jgdy, vmin=-100, vmax=100, cmap=gkr_col)
plt.show()

## 3.2 Estimando $T$

Você precisará implementar uma função que estime o *"tensor de orientação"* para uma região específica. \
Construa a função o mais geral possível, *i.e.*, que permita diferentes tamanhos de janela e também janelas não quadradas. \
Uma possível função protótipo é:

`estimar_T(Jgdx, Jgdy, x, y, window_size) -> T`

Entradas:
- `Jgdx`: $\frac{\partial }{\partial x}(J * g(\sigma))$
- `Jgdy`: $\frac{\partial }{\partial y}(J * g(\sigma))$
- `x`, `y`: Coordenadas horizontal e vertical do centro da janela
- `window_size`: Tupla contendo o tamanho da janela nos eixos $(x,y)$

Saída:
- `T`: Tensor de estrutura

In [ ]:
# Escreva aqui sua resposta

def estimar_T(Jgdx, Jgdy, x, y, window_size):
    half_window = ...
    max_y, max_x = ...
    lower_y = ...
    upper_y = ...
    lower_x = ...
    upper_x = ...
    
    x_window = Jgdx[lower_y:upper_y, lower_x:upper_x]
    y_window = Jgdy[lower_y:upper_y, lower_x:upper_x]
   
    T = np.zeros((2,2))
    
    T11 = np.sum(np.multiply(, ))
    T22 = np.sum(np.multiply(, ))
    T12 = np.sum(np.multiply(, ))
    T21 = ...
    
    T[0,0] = ...
    T[1,1] = ...
    T[0,1] = ...
    T[1,0] = ...
    
    return T

In [ ]:
T = estimar_T(Jgdx, Jgdy, 85, 120, (70, 40))

In [ ]:
import cv2 as cv2

Itemp = I.copy()
_ = cv2.rectangle(Itemp, (x, y), (x+window_size[0], y+window_size[1]), (255, 0, 0), 2)

plt.figure()
plt.imshow(Itemp)
plt.show()

## 3.3 Função de diferenças

Para atualizar o deslocamento com as equações do LK-tracker precisamos estimar o vetor $e$ para uma região específica. \
Implemente uma função que faça isso.
Faça esta função o mais geral possível, *i.e.*, que permita diferentes tamanhos de janela e também janelas não quadradas. \
Uma possível função protótipo é:

`estimar_e(Ig, Jg, Jgdx, Jgdy, x, y, window_size) -> e`

In [ ]:
# Escreva aqui sua resposta

def estimar_e(Ig, Jg, Jgdx, Jgdy, x, y, window_size):
    half_window = ...
    max_y, max_x = ...
    lower_y = ...
    upper_y = ...
    lower_x = ...
    upper_x = ...
    
    x_window = Jgdx[lower_y:upper_y, lower_x:upper_x]
    y_window = Jgdy[lower_y:upper_y, lower_x:upper_x]
   
    e = np.zeros((2,1))
    
    e1 = np.sum(np.multiply(, ))
    e2 = np.sum(np.multiply(, ))
    
    e[0] = e1
    e[1] = e2
    
    return e

In [ ]:
e = estimar_e(Ig, Jg, Jgdx, Jgdy, 85, 120, (70, 40))

## 3.4 Função de interpolação

Durante as iterações de gradiente descendente, é evidente que precisamos obter valores de intensidade para coordenadas de píxel não inteiras. \
Você deve implementar uma função que dê conta disso. \
Crie uma função que retorne valores de intensidade interpolados para todas as posições de sub-píxel especificadas por uma região de interesse. \
Você pode implementar mais de uma função de interpolação, porém não será necessário para passar esta tarefa.

**Dica**: Use [`scipy.interpolate.RectBivariateSpline`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RectBivariateSpline.html) para obter uma função de interpolação:

`imgc = RectBivariateSpline(np.arange(img.shape[0]), np.arange(img.shape[1]), img)`

**\* Passo de preparação**: Use `RectBivariateSpline` para definir um objeto de interpolação para uma imagem.
Use seu novo objeto para recortar regiões de interesse em uma imagem de teste.
Certifique-se de entender completamente a estrutura da chamada.

In [ ]:
# Escreva aqui sua resposta

import scipy

def interp_spline(xcoords, ycoords, Jg):
    ...
    
    return imgc

## 3.5 Finalizando o LK tracker

Agora que você tem todos os blocos constituintes (a função de derivadas regularizadas, a função do tensor de orientação, a função de diferenças e a função de interpolação) você está na capacidade de poder finalizar o seu LK tracker.
O que você precisa basicamente é resolver a ***equação do deslocamento*** e um laço de controle que atualize o vetor de deslocamento até que um ***critério de parada*** seja atingido, seja por um erro suficientemente pequeno, ou por número máximo de iterações alcançado.
Informe com uma mensagem caso um dos critérios for atingido.
Mostre na tela a evolução da estimativa do deslocamento a cada passo.
Uma possível função protótipo é:

`estimar_d(I, J, x, y, window_size, kernel_size, sigma, maxIter, minErr) -> d_total`

**Dica:** Use `np.linalg.solve` para resolver um sistema de equações lineares.

In [ ]:
# Escreva aqui sua resposta

def estimar_d(I, J, x, y, window_size, kernel_size, sigma, maxIter=100, minErr=0.01):
    d_total = np.zeros((2, 1))

    height, width = ...
    ycoords, xcoords = ...
    
    Ig, Jg, Jgdx, Jgdy = gradiente_regularizado(..., ..., ..., ...)
    Jgd = ...
    Jgdxd = ...
    Jgdyd = ...
    
    for step in range(maxIter):
        T = ...
        e = ...
        d = ...
        d_total = ...
        
        print('passo %02d          : ' % step, tuple(d_total.ravel()))
        if step == maxIter-1:
            print('Número máximo de iterações atingido')
        
        if np.linalg.norm(d) < minErr:
            print('Erro mínimo atingido')
            break
        
        xcoords_interp = np.arange(d_total[0], d_total[0] + width)
        ycoords_interp = np.arange(d_total[1], d_total[1] + height)
        
        Jgd   = interp_spline(ycoords, xcoords, Jg)  (ycoords_interp, xcoords_interp, grid=True)
        Jgdxd = interp_spline(ycoords, xcoords, Jgdx)(ycoords_interp, xcoords_interp, grid=True)
        Jgdyd = interp_spline(ycoords, xcoords, Jgdy)(ycoords_interp, xcoords_interp, grid=True)
    
    return d_total

Verifique que sua implementação funcione corretamente. Para isso, use a função `get_cameraman()` para obter `I`, `J` e `dTrue`.
`I` é a imagem original, `J` é uma versão deslocada da mesma imagem, e `dTrue` é um vetor descrevendo o deslocamento entre as imagens.
Rastreie uma região com `[altura, largura] = [70, 40]` centrada ao redor de `[linha, coluna] = [85, 120]`.
Ao testar sua implementação, seu deslocamento estimado $d$ deveria estar perto de `dTrue`, mesmo após a primeira iteração.
Verifique que a estimativa do deslocamento melhore ao realizar várias iterações.

**Perguntas:** Quais são os deslocamentos estimados após a primeira e segunda iteração? Qual é o deslocamento real?

In [ ]:
I, J, dTrue = lab3.get_cameraman()
d = estimar_d(I, J, 120, 85, (40, 70), 7, 0.5, maxIter=2)

print('Valor real d     :  (%.16f, %.16f)' % dTrue)
print('Valor estimado d : ', tuple(d.ravel()))

In [ ]:
# Escreva aqui sua resposta

# Referências

\[1\] Björn Johansson. Derivation of the Lucas-Kanade Tracker. 2007.